#### CELL-1 코드리뷰 

- <strong>NAME = ''</strong> -> 가져올 모델의 이름을 입력합니다 *전 과정에서 지정했던 이름으로 입력합니다


In [1]:
# Set Hyper Parameters
NAME = '' #모델의 이름을 입력합니다 예시: '12'

<br></br>

#### CELL-2 코드리뷰 

- <strong>코드리뷰는 코드 위에 있는 주석으로 대체합니다</strong> 

In [ ]:
import jajucha2
import torch
import numpy as np
import torchvision.transforms as transforms
from PIL import Image as Img
import cv2

# 인공지능 모델 가져오기
device = torch.device('cuda')  # GPU 장치를 사용하기 위해 CUDA 장치를 설정
model = jajucha2.ai.get_model(NAME)  # `jajucha2` 라이브러리에서 인공지능 모델을 가져옴
model.to(device)  # 모델을 CUDA 장치로 이동
model.eval()  # 모델을 평가 모드로 설정 (추론 시에만 사용하며, 학습과 관련된 연산을 비활성화)

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지를 224x224 크기로 조정 (모델에 입력하기 위해 고정된 크기로 맞춤)
    transforms.ToTensor(),  # 이미지를 PyTorch 텐서(tensor)로 변환 (이미지를 정규화하여 [0, 1] 범위의 값으로 변환하고, 채널 순서를 [H, W, C]에서 [C, H, W]로 변경)
])

# 초기 조향 및 속도 값 설정
steer = 0
speed = 0

# 메인 루프 시작
while True:
    image = jajucha2.camera.get_image('center')  # 중앙 카메라에서 이미지를 가져옴

    ## 인공지능 기반 신호등 감지
    image = Img.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # OpenCV의 BGR 이미지를 PIL의 RGB 이미지로 변환
    image = transform(image)  # 이미지 변환 (예: 크기 조정, 정규화 등)
    image = image.unsqueeze(0).to(device)  # 배치 차원을 추가하고 이미지를 CUDA 장치로 이동
    output = model(image)  # 인공지능 모델을 사용하여 이미지에 대한 예측 실행

    print(f"\r {output}", end="") # 모델의 출력값을 출력, 클래스의 갯수 만큼 리스트가 출력됨
    
    max_index = torch.argmax(output)  # 가장 높은 확률을 가진 클래스의 인덱스를 가져옴
    
    # 신호등이 빨간 불일 경우 (가정: 클래스 인덱스 1이 빨간 불을 의미)
    if max_index == 1:
        speed = 0  # 속도를 0으로 설정하여 정지
    
    # 설정된 조향과 속도를 차량 제어에 반영
    jajucha2.control.set_motor(steer, steer, speed)
